In [1]:
import json
import requests
from bs4 import BeautifulSoup as bts
import bs4
import pandas as pd
import numpy as np
import requests
import time

In [2]:
def getAndParseURL(url):
    result = requests.get(url,headers={"User-Agent":"Mozilla/5.0"})
    soup = bts(result.text, 'html.parser')
    return soup

In [3]:
html = getAndParseURL("https://www.imdb.com/search/title/?title_type=feature&user_rating=7.0,10.0&groups=top_1000&count=250")
"https://www.imdb.com/" + html.find('h3', class_='lister-item-header').a["href"]

'https://www.imdb.com//title/tt10872600/?ref_=adv_li_tt'

In [ ]:
def harvest_movies(movies, result):
    for movielink in movies:
        html = getAndParseURL(movielink)
        
        #name
        name = html.find('div', {'class': 'TitleBlock__TitleContainer-sc-1nlhx7j-1 jxsVNt'}).h1.text
        
        
        #year and runtime
        divv = html.find('div', {'class': 'TitleBlock__TitleMetaDataContainer-sc-1nlhx7j-2 hWHMKr'})
        metadata_container = divv.find("ul")
        metadata = metadata_container.findAll("li")
        metadata_length = len(metadata)

        if metadata_length==3:
            year = metadata[0].find("span").text
            runtime = metadata[2].text
        elif metadata_length==2:
            year = metadata[0].find("span").text
            runtime = metadata[1].text
        else:
            year = metadata[0].find("span").text
            runtime = np.NaN


        #gross
        gr = html.find('li', {'class': 'ipc-metadata-list__item BoxOffice__MetaDataListItemBoxOffice-sc-40s2pl-2 gwNUHl', 
                              'data-testid': 'title-boxoffice-cumulativeworldwidegross'})
        if gr is None:
            gross=np.NaN
        else:
            gross = gr.find("ul").findAll("li")[0].text.lstrip("$")


        #budget
        bud = html.find('li', {'data-testid': 'title-boxoffice-budget'})
        if bud is None:
            budget = np.NaN
        else:
            budget = bud.li.text


        #genres   
        gnr = html.find('div', { 'data-testid': 'genres'})
        if gnr is None:
            continue
            
        sp = gnr.findAll("span")
        if sp is None:
            sp = np.NaN
        elif len(sp)==3:
            genre1 = sp[0].text
            genre2 = sp[1].text
            genre3 = sp[2].text
        elif len(sp)==2:
            genre1 = sp[0].text
            genre2 = sp[1].text
            genre3 = "-"
        else:
            genre1 = sp[0].text
            genre2 = "-"
            genre3 = "-"


            
        #stars
        stars = html.findAll('li', {'class': "ipc-metadata-list__item ipc-metadata-list-item--link", 
                                    "data-testid": "title-pc-principal-credit"})[1]
        if stars is None:
            continue
            
        li_st = stars.findAll('li')
        if li_st is None:
            li_st = np.NaN

        elif len(li_st)==3:
            star1 = li_st[0].a.text
            star2 = li_st[1].a.text
            star3 = li_st[2].a.text

        elif len(li_st)==2:
            star1 = li_st[0].a.text
            star2 = li_st[1].a.text
            star3 = "-"

        else:
            star1 = li_st[0].a.text
            star2 = "-"
            star3 = "-"
        
        
        
        
        #company
        companyAll = html.find('li', {'data-testid': 'title-details-companies'})
        if companyAll is None:
            continue
            
        companyAll_div = companyAll.find('div')
        companyAll_ul = companyAll_div.find('ul')
       
        if len(companyAll_ul)==3:
            company1 = companyAll_ul.findAll('li')[0].text
            company2 = companyAll_ul.findAll('li')[1].text
            company3 = companyAll_ul.findAll('li')[2].text
        
        elif len(companyAll_ul)==2:
            company1 = companyAll_ul.findAll('li')[0].text
            company2 = companyAll_ul.findAll('li')[1].text
            company3 = "-"
            
        else:
            company1 = companyAll_ul.findAll('li')[0].text
            company2 = "-"
            company3 = "-"
        
        
        
        #rating
        rat = html.find('span', { 'class': 'AggregateRatingButton__RatingScore-sc-1ll29m0-1 iTLWoV'})
        if rat is None:
            rating=np.NaN
        else:
            rating=rat.text


        #user and critic reviews
        ul_ = html.find('ul', { 'class': 'ipc-inline-list ReviewContent__StyledInlineList-vlmc3o-0 hyrmRe baseAlt'})
        if ul_ is None:
            print("burda ul is None")
            continue

        li_ = ul_.findAll("li")
        if len(li_)<2:
            print("burda li <2")
            continue

        user_reviews = li_[0].find("span").span.text
        critic_reviews = li_[1].find("span").span.text


        #release date
        rel = html.find('li', { 'data-testid': 'title-details-releasedate'})
        if rel is None:
            release_date = np.NaN
        else:
            release_date = rel.find("ul").text


        #count of videos
        vid = html.find('div', {'class': 'Link__MediaLinkContentContainer-yyqs5y-2 rnVnr'})
        if vid is None:
            video = np.NaN
        else:
            video = vid.text.strip(" Videos")


        #count of photos   
        pho = html.find('a', {'data-testid': 'hero-media__photo-link'})
        if pho is None:
            photo = np.NaN
        else:
            photo = pho.text.strip(" Photos")


        #trailer duration
        dur = html.find('span', {'class': 'Slatestyles__Runtime-sc-1t1hgxj-7 jeEECx'})
        if dur is None:
            trailer_dur = np.NaN
        else:
            trailer_dur = dur.text
            
        
        #director
        director = html.find('a', {'class': 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'}).text

        
        
        #metascore
        meta = html.find('span', {'class': 'score-meta'})
        if meta is None:
            metascore = np.NaN
        else:
            metascore = meta.text
        
        
        
        result.append([name, year, runtime, budget, genre1, genre2, genre3, rating, user_reviews, 
                       critic_reviews, metascore, release_date, video, photo, trailer_dur, director, star1, star2, star3, company1, company2, company3, gross])
        time.sleep(2)

In [ ]:
result = []
for i in range(0,4):
    movies = []
    start = i*250+1
    html = getAndParseURL("https://www.imdb.com/search/title/?title_type=feature&user_rating=7.0,10.0&groups=top_1000&count=250&start=% s&sort=num_votes,desc" % start)
    for sonuc in html.findAll('h3', class_='lister-item-header'):
        movies.append("https://www.imdb.com/"+ sonuc.a["href"])
    harvest_movies(movies, result)
    # We used "start" inside the link to automatically change the number of pages (start=% s&sort=num_votes,desc" % start). 

column = ["name", "year", "runtime", "budget", "genre1", "genre2", "genre3", "rating", "user_reviews", 
          "critic_reviews", "metascore", "release_date", "count_videos", "count_photos", "trailer_duration", 
          "director", "star1", "star2", "star3", "company1", "company2", "company3", "gross"]

dfMatch = pd.DataFrame.from_records(result, columns=column)

In [ ]:
dfMatch

,name,year,runtime,budget,genre1,genre2,genre3,rating,user_reviews,critic_reviews,...,count_photos,trailer_duration,director,star1,star2,star3,company1,company2,company3,gross
0,The Shawshank Redemption,1994,2h 22m,"$25,000,000 (estimated)",Drama,-,-,9.3,9.7K,190,...,99+,2:11,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,Castle Rock Entertainment,-,-,"28,817,291"
1,The Dark Knight,2008,2h 32m,"$185,000,000 (estimated)",Action,Crime,Drama,9.0,7.7K,428,...,99+,0:33,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Warner Bros.,Legendary Entertainment,Syncopy,"1,005,973,645"
2,Inception,2010,2h 28m,"$160,000,000 (estimated)",Action,Adventure,Sci-Fi,8.8,4.4K,479,...,99+,1:45,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Warner Bros.,Legendary Entertainment,Syncopy,"836,836,967"
3,Fight Club,1999,2h 19m,"$63,000,000 (estimated)",Drama,-,-,8.8,4.1K,366,...,99+,2:00,David Fincher,Brad Pitt,Edward Norton,Meat Loaf,Fox 2000 Pictures,New Regency Productions,Linson Films,"101,209,702"
4,Forrest Gump,1994,2h 22m,"$55,000,000 (estimated)",Drama,Romance,-,8.8,2.8K,165,...,99+,3:54,Robert Zemeckis,Tom Hanks,Robin Wright,Gary Sinise,Paramount Pictures,The Steve Tisch Company,Wendy Finerman Productions,"678,226,133"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,In Cold Blood,1967,2h 14m,"$3,500,000 (estimated)",Biography,Crime,Drama,7.9,151,87,...,99+,2:55,Richard Brooks,Robert Blake,Scott Wilson,John Forsythe,Columbia Pictures,Pax Enterprises,-,NaN
996,Sullivan's Travels,1941,1h 30m,"$689,665 (estimated)",Adventure,Comedy,Drama,7.9,165,106,...,41,1:51,Preston Sturges,Joel McCrea,Veronica Lake,Robert Warwick,Paramount Pictures,-,-,"10,249"
997,Yume,1990,1h 59m,"$12,000,000 (estimated)",Drama,Fantasy,-,7.8,134,54,...,84,1:44,Akira Kurosawa,Akira Terao,Mitsuko Baishô,Toshie Negishi,Warner Bros.,Akira Kurosawa USA,-,"2,970,161"
998,Nashville,1975,2h 40m,"$2,200,000 (estimated)",Comedy,Drama,Music,7.7,187,109,...,99+,2:11,Robert Altman,Keith Carradine,Karen Black,Ronee Blakley,ABC Entertainment,American Broadcasting Company (ABC),Paramount Pictures,"9,991,258"


In [ ]:
dfMatch.to_csv("moviemedium.csv")